In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

In [2]:
torch.__version__

'1.10.1'

In [3]:
def hamming_score(y_pred, y_true):
    out = ((y_pred & y_true).sum(dim=1) / (y_pred | y_true).sum(dim=1)).mean()
    if out.isnan():
        out = torch.tensor(1.0)
    return out

In [4]:
class SVD(nn.Module):
    """
    Singular value decomposition layer
    
    Parameters
    ----------
    compute_uv: bool
        Control whether to compute `U` and `V`.
    
    Examples
    --------
    >>> A = torch.rand(126, 100, 20).to('cuda')
    >>> U, S, V = SVD(compute_uv=True)(A)
    >>> A_ = torch.matmul(U, torch.matmul(S, V.transpose(-1, -2)))
    >>> print(torch.dist(A_, A))
    """
    def __init__(self, compute_uv=True):
        super(SVD, self).__init__()
        self.compute_uv = compute_uv
    
    def forward(self, A):
        """
        Inputs
        ------
        A: [b, m, n]
        
        Outputs
        -------
        U: [b, m, n]
        S: [b, n, n]
        V: [b, n, n]
        """
        return self.svd_(A)
        
    def svd_(self, A):
        with torch.no_grad():
            if self.compute_uv:
                U, S, V = torch.svd(A)
                S = torch.diag_embed(S)
                return U, S, V
            elif not self.compute_uv:
                _, S, _ = torch.svd(A, compute_uv=False)
                S = torch.diag_embed(S)
                return S
            
def singular_value_cumsum(S):
    """
    S: [b, n, n]
        S is a diagonal matrix whose off-diagonal entries are all equal to zero.
    """
    numerator = torch.diagonal(S, dim1=-2, dim2=-1).cumsum(dim=1)
    denominator = torch.diagonal(S, dim1=-2, dim2=-1).sum(dim=1, keepdim=True)
    return torch.div(numerator, denominator)

In [5]:
A = torch.rand(4, 512, 10)
A = A.to('cuda')
U, S, V = SVD()(A)

In [6]:
print(U.shape, S.shape, V.shape)

torch.Size([4, 512, 10]) torch.Size([4, 10, 10]) torch.Size([4, 10, 10])
